In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import DenseNet121
import os
import cv2
from numpy import asarray
from PIL import Image
from functools import partial
from keras.applications import imagenet_utils

Using TensorFlow backend.


In [6]:
### Organizing classes into training, validation, and test
# print("Organizing datasets")
# masked_imgs = os.listdir('datasets/compiled/with_mask_copy')
# no_mask_imgs = os.listdir('datasets/compiled/without_mask_copy')

# i = 0
# for img in masked_imgs:
# 	if i < int(0.80*len(masked_imgs)):
# 		os.rename('datasets/compiled/with_mask_copy/' + img, 'datasets/compiled/train/mask/' + img)
# 		i += 1
# 	else:
# 		os.rename('datasets/compiled/with_mask_copy/' + img, 'datasets/compiled/validation/mask/' + img)
# 		i += 1

# j = 0
# for img in no_mask_imgs:
# 	if j < int(0.80*len(no_mask_imgs)):
# 		os.rename('datasets/compiled/without_mask_copy/' + img,'datasets/compiled/train/no-mask/' + img)
# 		j += 1
# 	else:
# 		os.rename('datasets/compiled/without_mask_copy/' + img, 'datasets/compiled/validation/no-mask/' + img)
# 		j += 1

# print("Images reorganized")

In [7]:
epochs = 1
batch_size = 16

In [8]:
# imagenet_mode_preprocessing_function = partial(imagenet_utils.preprocess_input, mode="torch")

In [20]:
train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="nearest",
    horizontal_flip=True,
    rescale=1./255)

validation_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="nearest",
    horizontal_flip=True,
    rescale=1./255)

In [21]:
train_generator = train_datagen.flow_from_directory(
        batch_size=batch_size,
		directory='datasets/compiled/train/',
        target_size=(224, 224), 
        classes = ['no-mask','mask'],
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        batch_size=batch_size,
        directory='datasets/compiled/validation/',
        target_size=(224, 224), 
        classes = ['no-mask','mask'],
        class_mode='categorical')

Found 3065 images belonging to 2 classes.
Found 768 images belonging to 2 classes.


In [22]:
### Pre-trained Model (DenseNet121 trained on Imagenet)
model = tf.keras.applications.DenseNet121(include_top=False,weights='imagenet',input_shape=(224,224,3),classes=2)

In [23]:
# Transfer Learning
for i in model.layers:
  i.trainable = False

In [24]:
global_avg = tf.keras.layers.GlobalAveragePooling2D()(model.output)
flatten = tf.keras.layers.Flatten()(global_avg)
# drop_out = tf.keras.layers.Dropout(0.4)(flatten)
out = tf.keras.layers.Dense(2,activation='softmax')(flatten)
densenet = tf.keras.Model(inputs=[model.input],outputs=[out])

In [25]:
densenet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [26]:
densenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=["accuracy"])

In [27]:
history = densenet.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

192/192 [==============================] - 355s 2s/step - loss: 0.0838 - accuracy: 0.9726 - val_loss: 0.0395 - val_accuracy: 0.9909


In [28]:
densenet.save('densenet121_detection_model.h5')

In [29]:
TI = Image.open('datasets/compiled/mask_test.jpeg')
test_img = np.asarray(TI)
test = test_img.copy()
test.resize(1,224,224,3)
print(test.shape)
print(densenet.predict(test))

(1, 224, 224, 3)
[[2.3569877e-10 1.0000000e+00]]


In [2]:
load_model = keras.models.load_model('densenet121_detection_model.h5')

In [3]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [4]:
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
cv2.namedWindow("COVID Mask Detection Video Feed")
webcam = cv2.VideoCapture(0) 

classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    rval, im = webcam.read()
    im=cv2.flip(im,1,1)
    
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))
 
    faces = classifier.detectMultiScale(mini)

    for f in faces:
        (x, y, w, h) = [v * size for v in f] 
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(224,224))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,224,224,3))
        reshaped = np.vstack([reshaped])
        result=load_model.predict(reshaped)
        print(result)
        if result[0][0] > result[0][1]:
            percent = round(result[0][0]*100,2)
        else:
            percent = round(result[0][1]*100,2)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label] + " " + str(percent) + "%", (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    if im is not None:   
        cv2.imshow('COVID Mask Detection Video Feed', im)
    key = cv2.waitKey(10)
    
    # Exit
    if key == 27: #The Esc key
        break
        
# Stop video
webcam.release()

# Close all windows
cv2.destroyAllWindows()

[[9.9962449e-01 3.7554206e-04]]
[[9.999335e-01 6.653704e-05]]
[[9.9994779e-01 5.2200623e-05]]
[[9.9994862e-01 5.1434712e-05]]
[[9.9994540e-01 5.4599273e-05]]
[[9.9990714e-01 9.2871524e-05]]
[[9.9995899e-01 4.0974955e-05]]
[[9.999298e-01 7.017693e-05]]
[[9.9991286e-01 8.7094588e-05]]
[[9.999752e-01 2.483037e-05]]
[[9.9993324e-01 6.6803557e-05]]
[[9.99895334e-01 1.04713596e-04]]
[[9.9994063e-01 5.9387745e-05]]
[[9.9996734e-01 3.2698525e-05]]
[[9.9994826e-01 5.1698138e-05]]
[[9.9995315e-01 4.6792986e-05]]
[[9.9997342e-01 2.6598025e-05]]
[[9.9996936e-01 3.0686391e-05]]
[[9.9993312e-01 6.6917935e-05]]
[[9.9993348e-01 6.6463974e-05]]
[[9.9993753e-01 6.2499334e-05]]
[[9.99897122e-01 1.02852704e-04]]
[[9.9995470e-01 4.5343266e-05]]
[[9.999517e-01 4.822316e-05]]
[[9.9995553e-01 4.4436358e-05]]
[[9.999578e-01 4.222377e-05]]
[[9.9959236e-01 4.0760014e-04]]
[[9.999218e-01 7.814359e-05]]
[[9.999131e-01 8.695003e-05]]
[[9.9994624e-01 5.3809665e-05]]
[[9.9991202e-01 8.8017594e-05]]
[[9.9995184e-01 4.

[[2.000466e-08 1.000000e+00]]
[[1.2380091e-08 1.0000000e+00]]
[[2.4261984e-08 1.0000000e+00]]
[[4.5979967e-09 1.0000000e+00]]
[[9.305054e-09 1.000000e+00]]
[[5.131021e-09 1.000000e+00]]
[[9.762965e-09 1.000000e+00]]
[[1.3168438e-08 1.0000000e+00]]
[[1.150282e-07 9.999999e-01]]
[[2.0866008e-08 1.0000000e+00]]
[[3.2988192e-09 1.0000000e+00]]
[[4.2550234e-09 1.0000000e+00]]
[[5.860838e-09 1.000000e+00]]
[[3.9514870e-07 9.9999964e-01]]
[[1.3686611e-08 1.0000000e+00]]
[[9.486878e-09 1.000000e+00]]
[[2.3783537e-09 1.0000000e+00]]
[[5.2958966e-09 1.0000000e+00]]
[[4.3762562e-09 1.0000000e+00]]
[[2.1497058e-08 1.0000000e+00]]
[[4.4691033e-09 1.0000000e+00]]
[[9.099541e-10 1.000000e+00]]
[[9.918733e-09 1.000000e+00]]
[[1.2255361e-08 1.0000000e+00]]
[[9.132264e-10 1.000000e+00]]
[[3.4318783e-09 1.0000000e+00]]
[[1.935349e-08 1.000000e+00]]
[[2.2867269e-08 1.0000000e+00]]
[[2.6940719e-09 1.0000000e+00]]
[[1.1481066e-09 1.0000000e+00]]
[[7.1078783e-09 1.0000000e+00]]
[[5.0262816e-09 1.0000000e+00]

[[9.994312e-01 5.687371e-04]]
[[9.9924976e-01 7.5023819e-04]]
[[9.990055e-01 9.945040e-04]]
[[9.9957186e-01 4.2817235e-04]]
[[9.9957222e-01 4.2773454e-04]]
[[0.9989868  0.00101325]]
[[9.9961287e-01 3.8715528e-04]]


KeyboardInterrupt: 